In [19]:
import pandas as pd
import os
import torch
import sqlite3
from sqlite3 import Error

In [ ]:
device = 'cuda'
model_path = 'gandalf.pt'
latex = """"""

for subdir in os.listdir('data'):
    data_dir = f'data/{subdir}'
    ids  = [file.replace('.edf','') for file in os.listdir(data_dir) if file.endswith('.edf')]
    edfs = [file for file in os.listdir(data_dir) if file.endswith('.edf')]
    zdbs = [file for file in os.listdir(data_dir) if file.endswith('.zdb')]
    print(data_dir,ids)
    if len(ids) != len(edfs) or len(ids) != len(zdbs) or len(edfs) != len(zdbs):
        print('big problem')
    id_to_zdb_filename = {}
    for id in ids:
        corresponding_zdb_filename_for_id = [zdb for zdb in zdbs if id in zdb]
        if len(corresponding_zdb_filename_for_id) != 1:
            print('big problem')
        id_to_zdb_filename[id] = corresponding_zdb_filename_for_id[0]
    for id in ids:
        zdb_filename = id_to_zdb_filename[id]
        edf_filename = f'{id}.edf'
        print(id)
        if id == "24-Jun-B1":
            eeg_ch_name = 'EEG 2'
        else:
            eeg_ch_name = 'EEG 1'

        y_pred = pd.read_csv(f'{data_dir}/{id}_y_pred.csv')
        y_pred = pd.DataFrame(y_pred).astype('str')
        y_pred.loc[y_pred['y_pred'] == '0','y_pred'] = 'P'
        y_pred.loc[y_pred['y_pred'] == '1','y_pred'] = 'S'
        y_pred.loc[y_pred['y_pred'] == '2','y_pred'] = 'W'


        rename_dict = {'W':'Sleep-Wake', 'S':'Sleep-SWS', 'P':'Sleep-Paradoxical', 'X':''}
        offset = 10e7 #epoch time period

        zdb_path = f'{data_dir}/{zdb_filename}'
        try:
            conn = sqlite3.connect(zdb_path)
        except Error as e:
            print(e)

        # get recording start stop
        cur = conn.cursor()
        query = "SELECT value FROM internal_property WHERE key='RecordingStart'"
        cur.execute(query)
        result = cur.fetchall()
        recording_start = int(result[0][0])
        query = "SELECT value FROM internal_property WHERE key='RecordingStop'"
        cur.execute(query)
        result = cur.fetchall()
        recording_stop = int(result[0][0])
        length_ns = recording_stop - recording_start # ns
        length_s = length_ns * 1e-7 # s
        hh = length_s // 3600
        mm = (length_s % 3600) // 60
        ss = ((length_s % 3600) % 60)
        print(hh,mm,ss,length_s)
        print(recording_start)
        print(recording_stop)

        #drop this table - creates issues
        query = "DROP TABLE IF EXISTS temporary_scoring_marker;"
        cur.execute(query)

        query = "SELECT name FROM sqlite_master WHERE type='table';"
        cur.execute(query)

        tables = [table[0] for table in cur.fetchall()]

        if 'scoring_revision' not in tables:
            query = "CREATE TABLE scoring_revision (id INTEGER PRIMARY KEY, name TEXT, is_deleted INTEGER(1), tags TEXT, version INTEGER(8), owner TEXT, date_created INTEGER(8));"
            cur.execute(query)

        if 'scoring_marker' not in tables:
            query = "CREATE TABLE scoring_marker (id INTEGER PRIMARY KEY, starts_at INTEGER(8), ends_at INTEGER(8), notes TEXT, type TEXT, location TEXT, is_deleted INTEGER(1), key_id INTEGER);"
            cur.execute(query)

        if 'scoring_comment' not in tables:
            query = "CREATE TABLE scoring_comment (id INTEGER PRIMARY KEY, category TEXT, key TEXT, value TEXT);"
            cur.execute(query)

        if 'scoring_key' not in tables:
            query = "CREATE TABLE scoring_key (id INTEGER PRIMARY KEY, date_created INTEGER(8), name TEXT, owner TEXT, type TEXT);"
            cur.execute(query)

        if 'scoring_revision_to_comment' not in tables:
            query = "CREATE TABLE scoring_revision_to_comment (revision_id INTEGER(8), comment_id INTEGER(8));"
            cur.execute(query)

        if 'scoring_revision_to_key' not in tables:
            query = "CREATE TABLE scoring_revision_to_key (revision_id INTEGER(8), key_id INTEGER(8));"
            cur.execute(query)

        # # delete first score before adding machine data
        # query = "DELETE FROM scoring_marker;"
        # cur.execute(query)

        # #delete first score before adding machine data
        # query = "DELETE FROM scoring_revision;"
        # cur.execute(query)

        # #delete first score before adding machine data
        # query = "DELETE FROM scoring_key;"
        # cur.execute(query)

        # #delete first score before adding machine data
        # query = "DELETE FROM scoring_revision_to_key;"
        # cur.execute(query)

        query = f"""
            INSERT INTO scoring_revision 
            (name, is_deleted, version, date_created)
            VALUES 
            ('LSTM', 0, 0, {recording_start});
            """ 
        cur.execute(query)

        query = f"""
            INSERT INTO scoring_key 
            (date_created, type)
            VALUES 
            ({recording_start},'Automatic');
            """ 
        cur.execute(query)

        query = f"""
            INSERT INTO scoring_revision_to_key 
            (revision_id, key_id)
            VALUES 
            (1,1);
            """
        cur.execute(query)

        #get keyid of scoring
        query = "SELECT MAX(id) FROM scoring_revision WHERE name='LSTM'"
        cur.execute(query)
        keyid = cur.fetchall()[0][0]

        start_time = recording_start - (recording_start % 100000000)
        stop_time = 0
        # #insert new epochs with scoring into the table
        for i in range(len(y_pred)):
            #calculate epoch
            if i != 0:
                start_time = stop_time
            stop_time = start_time+offset
            score = rename_dict[y_pred.at[i,'y_pred']]
            #insert epoch
            query = f"""
                    INSERT INTO scoring_marker 
                    (starts_at, ends_at, notes, type, location, is_deleted, key_id)
                    VALUES 
                    ({start_time}, {stop_time}, '', '{score}', '', 0, {keyid});
                    """ 
            cur.execute(query)

        conn.commit()
        conn.close()


data/24-Jun PD49 ['24-Jun-D2', '24-Jun-A1', '24-Jun-D1', '24-Jun-A2', '24-Jun-B2', '24-Jun-B1']
24-Jun-D2
70.0 57.0 42.0 255462.0
638597684610000000
638600239230000000
24-Jun-A1
43.0 40.0 7.0 157207.0
638597684610000000
638599256680000000
24-Jun-D1
70.0 57.0 42.0 255462.0
638597684610000000
638600239230000000
24-Jun-A2
70.0 57.0 42.0 255462.0
638597684610000000
638600239230000000
24-Jun-B2
70.0 57.0 42.0 255462.0
638597684610000000
638600239230000000
24-Jun-B1
70.0 57.0 42.0 255462.0
638597684610000000
638600239230000000
data/24-Jun PD28 ['24-Jun-D2', '24-Jun-A1', '24-Jun-D1', '24-Jun-A2', '24-Jun-B2', '24-Jun-B1']
24-Jun-D2
73.0 28.0 16.0 264496.0
638579458310000000
638582103270000000
24-Jun-A1
73.0 28.0 16.0 264496.0
638579458310000000
638582103270000000
24-Jun-D1
73.0 28.0 16.0 264496.0
638579458310000000
638582103270000000
24-Jun-A2
73.0 28.0 16.0 264496.0
638579458310000000
638582103270000000
24-Jun-B2
73.0 28.0 16.0 264496.0
638579458310000000
638582103270000000
24-Jun-B1
73.0 28

In [17]:
zdb_filename

'EKyn Sleep and Sleep Dep 24-Jun PD49.24-Jun-B1.20240820203421.zdb'

y_pred
W         14472
S         10331
P           744
Name: count, dtype: int64